# News Stance and Bias Detector
#### Name : Kessyl Stelorra

#### Description :
This project collects recent news articles related to a chosen topic (e.g., “Trump Tariff”), then uses IBM Granite Instruct, a large language model, to automatically analyze each article’s stance and bias.
The final output is a structured dataset summarizing how different media sources portray the topic.

#### Workflow :
1. The code uses NewsAPI to search for the latest and most popular articles about a chosen topic, such as “Trump Tariff”.
2. Articles that are too short or have missing content are removed to make sure the analysis is accurate.
3. The text of each article is sent to the IBM Granite Instruct AI model. The AI reads the article and determines: whether the article is Pro, Against, or Neutral about the topic (stance), and whether the tone is Objective, Emotional, Partisan, or Mixed (bias).
4. The final results are shown in a table with each article’s stance, bias, and a short explanation of why it was classified that way.

In [8]:
# Use NEWSAPI to get news data

import requests
import pandas as pd

url = ('https://newsapi.org/v2/everything?'
       'q=Trump+AND+Tariff&'
       'from=2025-09-19&'
       'sortBy=popularity&'
       'pageSize=50&'
       'sources=bbc-news&'
       'apiKey=59b1b70aef89462880affcaf16218dbc')

response = requests.get(url)
articles = response.json()

In [9]:
# Arrange the data into dataframe

news_list = articles["articles"]

df = pd.DataFrame(news_list)
df["source"] = df["source"].apply(lambda x: x["name"])

df = df[["source", "author", "title", "description", "content", "url", "publishedAt"]]

df = df.dropna(subset=["content"])
df = df[df["content"].str.len() > 50]

df.head()

,source,author,title,description,content,url,publishedAt
0,BBC News,None,Gaza prepares for the first phase of Trump pea...,Donald Trump says Hamas is gathering the Israe...,Donald Trump says Hamas is gathering the Israe...,https://www.bbc.co.uk/programmes/w172zss9mnyx96d,2025-10-11T13:19:00Z
1,BBC News,Bbc News,Donald Trump's 20-point Gaza peace plan in full,European and Middle Eastern leaders have welco...,1. Gaza will be a deradicalised terror-free zo...,https://www.bbc.co.uk/news/articles/c70155nked7o,2025-09-30T06:12:28Z
2,BBC News,Natalie Sherman,Trump threatens to cancel meeting with Xi,"Trump also threatened a ""massive"" tariff incre...",As well as tightening rules for exports of rar...,https://www.bbc.co.uk/news/articles/cn4wkd7729po,2025-10-10T16:51:01Z
3,BBC News,Stephen Mcdonell,China accuses US of 'double standards' over ta...,"Beijing says it could introduce ""countermeasur...",Trump's comments on Friday rattled financial m...,https://www.bbc.co.uk/news/articles/cn828kg8rmzo,2025-10-12T14:42:53Z
4,BBC News,Osmond Chia,"Trump announces new tariffs on trucks, drugs a...",The US president said the move aims to help pr...,Trump said the tariffs on heavy trucks were to...,https://www.bbc.co.uk/news/articles/crkjreprp3po,2025-09-26T00:42:11Z


In [10]:
# Install Replicate

!pip install langchain_community
!pip install replicate

In [11]:
# Command to analyze the data

from langchain_community.llms import Replicate
import os
from google.colab import userdata

api_token = userdata.get('IBMCapstone')
os.environ["REPLICATE_API_TOKEN"] = api_token

llm = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    model_kwargs={"temperature": 0.2, "max_new_tokens": 300}
)

def analyze(text):
    prompt = f"""
    Analyze the following news article and identify:
    1. The stance of the article (Pro / Against / Neutral) towards the main topic.
    2. The bias of the article (Objective / Emotional / Partisan / Mixed).
    3. A brief explanation (max 2 sentences).

    Return your result in strict JSON format with keys: stance, bias, explanation.

    ARTICLE:
    {text}
    """
    try:
        result = llm.invoke(prompt)
        return result
    except Exception as e:
        return str(e)

In [12]:
# Command to analyze and arrange the data

import time
import json
import pandas as pd

def analyze2(text):
    try:
        result = analyze(text)
        data = json.loads(result)
        return data
    except Exception as e:
        print("Error:", e)
        return {"stance": None, "bias": None, "explanation": None}

In [13]:
# Analyze the data

results = []
for i, article in enumerate(df["content"]):
    print(f"Analyzing {i+1}/{len(df)}...")
    res = analyze2(article)
    results.append(res)
    time.sleep(3)

Analyzing 1/8...
Analyzing 2/8...
Error: Expecting value: line 1 column 1 (char 0)
Analyzing 3/8...
Error: Expecting value: line 1 column 1 (char 0)
Analyzing 4/8...
Analyzing 5/8...
Error: Expecting value: line 1 column 1 (char 0)
Analyzing 6/8...
Error: Expecting value: line 1 column 1 (char 0)
Analyzing 7/8...
Analyzing 8/8...
Error: Expecting value: line 1 column 1 (char 0)


In [14]:
# Filter the results

valid_results = [r for r in results if r["stance"] is not None]
results_df = pd.DataFrame(valid_results)
df_valid = df.head(len(results_df)).reset_index(drop=True)

final_df = pd.concat([df_valid, results_df], axis=1)
final_df = final_df[["title", "url", "stance", "bias", "explanation"]]
final_df.head()

,title,url,stance,bias,explanation
0,Gaza prepares for the first phase of Trump pea...,https://www.bbc.co.uk/programmes/w172zss9mnyx96d,Neutral,Mixed,The article presents Donald Trump's statement ...
1,Donald Trump's 20-point Gaza peace plan in full,https://www.bbc.co.uk/news/articles/c70155nked7o,Against,Emotional,The article expresses a negative stance toward...
2,Trump threatens to cancel meeting with Xi,https://www.bbc.co.uk/news/articles/cn4wkd7729po,Against,Emotional,The article reflects an 'Against' stance towar...
